In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import nan

In [ ]:
filepath="/project/haggis/ERA5/mon/Qfluxes/"
dat = xr.open_mfdataset(filepath+"Qfluxes*.nc")
dat = dat.sel(time=slice("1980-01","2020-12"))

dp = xr.DataArray(np.zeros([dat.time.size, dat.level.size, dat.lat.size, dat.lon.size]),
                 coords=dat.Q.coords, dims=dat.Q.dims, name='dp')
dp[:,0,:,:] = dat.level.isel(level=0)-0 + (dat.level.isel(level=1) - dat.level.isel(level=0))/2.
for ip in np.arange(1, dat.level.size-1,1):
    dp[:,ip,:,:] = ( (dat.level.isel(level=ip)-dat.level.isel(level=ip-1))/2. + (dat.level.isel(level=ip+1)-dat.level.isel(level=ip))/2. )
dp[:,dat.level.size-1,:,:] = (dat.level.isel(level=dat.level.size-1) - dat.level.isel(level=dat.level.size-2))/2.

# set levels below the surface to NaNs
dp = dp.where( dat.ps > dat.level*100, nan)
dp = dp.load()

minps = np.min(dat.ps)
levdif = dat.level*100. - np.array(minps)
levminsearch = np.max(np.argwhere(np.array(levdif) < 0))

for ilev in np.arange(levminsearch, dat.level.size,1):
    dp1d = dp.isel(level=ilev)
    dp1d = dp1d.load()
    if (ilev == dp.level.size-1):
        dp1d = dp1d.where( xr.ufuncs.isnan(dp.isel(level=ilev)),
                         (dp.level.isel(level=ilev)-dp.level.isel(level=ilev-1))/2. + (dat.ps/100.) - dp.level.isel(level=ilev))
    else:
        dp1d = dp1d.where( ~(xr.ufuncs.isnan(dp.isel(level=ilev+1)) & xr.ufuncs.isfinite(dp.isel(level=ilev))),
                         (dp.level.isel(level=ilev)-dp.level.isel(level=ilev-1))/2. + dat.ps/100. - dp.level.isel(level=ilev))
    dp[:,ilev,:,:] = dp1d
    
uqdp = dp*dat.uq
vqdp = dp*dat.vq
qdp = dp*dat.Q

uqdpsum = uqdp.sum(dim='level', skipna=True)
vqdpsum = vqdp.sum(dim='level', skipna=True)
qdpsum = qdp.sum(dim='level', skipna=True)

uqdpsum = uqdpsum.load()
vqdpsum = vqdpsum.load()
qdpsum = qdpsum.load()

In [ ]:
latrad = (dat.lat/180.)*np.pi
lonrad = (dat.lon/180.)*np.pi
a = 6.371e6

latradarray = np.tile(latrad,qdp.time.size*qdp.lon.size)
latradarray = np.reshape(latradarray,[qdp.time.size, qdp.lon.size, qdp.lat.size])
latradarray = np.moveaxis(latradarray, 1, 2)

duqdx = (1./(a*np.cos(latradarray)))*np.gradient(uqdpsum, lonrad, axis=2)
dvqdy = (1./(a*np.cos(latradarray)))*np.gradient(vqdpsum*np.cos(latradarray), latrad, axis=1)

q_flux_conv = (-1.*duqdx -dvqdy)*(100./(1000.*9.8))*86400.*1000.
q_flux_conv = xr.DataArray(q_flux_conv, coords=[dat.Q.time, dat.Q.lat, dat.Q.lon], dims=['time','lat','lon'], name='q_flux_conv')
q_flux_conv = q_flux_conv.where(np.abs(q_flux_conv) < 1e10, nan)
q_flux_conv = q_flux_conv.load()
q_flux_conv_clim = q_flux_conv.mean('time').load()
q_flux_conv_clim = q_flux_conv_clim.where( np.abs(q_flux_conv_clim) < 1e5,nan)